---

_You are currently looking at **version 1.1** of this notebook. To download notebooks and datafiles, as well as get help on Jupyter notebooks in the Coursera platform, visit the [Jupyter Notebook FAQ](https://www.coursera.org/learn/python-text-mining/resources/d9pwm) course resource._

---

# Assignment 1

In this assignment, you'll be working with messy medical data and using regex to extract relevant infromation from the data. 

Each line of the `dates.txt` file corresponds to a medical note. Each note has a date that needs to be extracted, but each date is encoded in one of many formats.

The goal of this assignment is to correctly identify all of the different date variants encoded in this dataset and to properly normalize and sort the dates. 

Here is a list of some of the variants you might encounter in this dataset:
* 04/20/2009; 04/20/09; 4/20/09; 4/3/09
* Mar-20-2009; Mar 20, 2009; March 20, 2009;  Mar. 20, 2009; Mar 20 2009;
* 20 Mar 2009; 20 March 2009; 20 Mar. 2009; 20 March, 2009
* Mar 20th, 2009; Mar 21st, 2009; Mar 22nd, 2009
* Feb 2009; Sep 2009; Oct 2010
* 6/2008; 12/2009
* 2009; 2010

Once you have extracted these date patterns from the text, the next step is to sort them in ascending chronological order accoring to the following rules:
* Assume all dates in xx/xx/xx format are mm/dd/yy
* Assume all dates where year is encoded in only two digits are years from the 1900's (e.g. 1/5/89 is January 5th, 1989)
* If the day is missing (e.g. 9/2009), assume it is the first day of the month (e.g. September 1, 2009).
* If the month is missing (e.g. 2010), assume it is the first of January of that year (e.g. January 1, 2010).
* Watch out for potential typos as this is a raw, real-life derived dataset.

With these rules in mind, find the correct date in each note and return a pandas Series in chronological order of the original Series' indices.

For example if the original series was this:

    0    1999
    1    2010
    2    1978
    3    2015
    4    1985

Your function should return this:

    0    2
    1    4
    2    0
    3    1
    4    3

Your score will be calculated using [Kendall's tau](https://en.wikipedia.org/wiki/Kendall_rank_correlation_coefficient), a correlation measure for ordinal data.

*This function should return a Series of length 500 and dtype int.*

In [1]:
import pandas as pd

doc = []
with open('dates.txt') as file:
    for line in file:
        doc.append(line)

df = pd.Series(doc)
df.head(10)

0         03/25/93 Total time of visit (in minutes):\n
1                       6/18/85 Primary Care Doctor:\n
2    sshe plans to move as of 7/8/71 In-Home Servic...
3                7 on 9/27/75 Audit C Score Current:\n
4    2/6/96 sleep studyPain Treatment Pain Level (N...
5                    .Per 7/06/79 Movement D/O note:\n
6    4, 5/18/78 Patient's thoughts about current su...
7    10/24/89 CPT Code: 90801 - Psychiatric Diagnos...
8                         3/7/86 SOS-10 Total Score:\n
9             (4/10/71)Score-1Audit C Score Current:\n
dtype: object

In [2]:
import calendar

df.replace(to_replace="(.*)Janaury(.*)", value="\\1January\\2", regex=True, inplace=True)
df.replace(to_replace="(.*)Decemeber(.*)", value="\\1December\\2", regex=True, inplace=True)

# Capture month/day/year
mdy = r'(?P<date>(?P<month>(0?\d)|(1[012]))[/-](?P<day>(0?\d)|([12]\d)|(3[01]))[/-](?P<year>((19)|(20))?\d{2})(?![\d/-]))'
mdy_extracted = df.str.extract(mdy)

# Capture month day year
month_abbrs = '|'.join(['({})'.format(m) for m in calendar.month_abbr[1:13]])
months = '|'.join(['({})'.format(m) for m in calendar.month_name[1:13]])
m = month_abbrs + '|' + months
md = r'(?P<date>(?P<month>'+ m + ')(-|(\.? ))(?P<day>(0?\d)|([12]\d)|(3[01]))((st)|(th)|(nd)|(rd))?(-|(,? ))(?P<year>((19)|(20))?\d{2})(?![\d/-]))'
md_extracted = df.str.extract(md)

# Capture day month year
dm = r'(?P<date>(?P<day>(0?\d)|([12]\d)|(3[01]))((st)|(th)|(nd)|(rd))? (?P<month>'+ m + ')(-|([.,]? ))(?P<year>((19)|(20))?\d{2})(?![\d/-]))'
dm_extracted = df.str.extract(dm)

# Capture month year
# I have to use a negative lookbehind on '\d ' so I don't recapture day month year.
# I hope all those month day years match '\d ' before month and year, and don't have wacky
# things like the 1st or whatever. If they do my lookbehind will not prevent their matches.
my = r'(?<!\d )(?P<date>(?P<month>'+ m + ')([.,]? )|/(?P<year>((19)|(20))\d{2}))'
my_extracted = df.str.extract(my)

# Capture month/year
msy = r'(?<![\d/])(?P<date>(?P<month>(0?\d)|(1[012]))[/-](?P<year>((19)|(20))?\d{2})(?![\d/]))'
msy_extracted = df.str.extract(msy)

# Capture just year
justyear = r'\s(?P<date>(?P<year>((19)|(20))\d{2}))(?![\d/-])'
jy_extracted = df.str.extract(justyear)

# One-by-one, find out the ranges of each type of date.
# Make sure a given regex works for each date in its range.
# In our final answer we will hard-code the ranges and concatenate
# mdy_extracted.loc[~mdy_extracted['date'].isna()] # 0:124
# dm_extracted.loc[~dm_extracted['date'].isna()] # 125:193
# md_extracted.loc[~md_extracted['date'].isna()] # 194:227
# my_extracted.loc[~my_extracted['date'].isna()] # 228:342
# msy_extracted.loc[~msy_extracted['date'].isna()] # 343:454, extras
# jy_extracted.loc[~jy_extracted['date'].isna()] # 455:499, extras

In [3]:
def date_sorter():
    import calendar
    import numpy as np

    # Clean typos
    for bad, good in {"Janaury": "January", "Decemeber": "December"}.items():
        df.replace(to_replace="(.*)"+bad+"(.*)", value="\\1"+good+"\\2", regex=True, inplace=True)

    month_abbrs = '|'.join(['({})'.format(m) for m in calendar.month_abbr[1:13]])
    months = '|'.join(['({})'.format(m) for m in calendar.month_name[1:13]])
    m = month_abbrs + '|' + months

    regex_and_range = [
        (r'(?P<date>(?P<month>(0?\d)|(1[012]))[/-](?P<day>(0?\d)|([12]\d)|(3[01]))[/-](?P<year>((19)|(20))?\d{2}))', 0, 124), # mm/dd/yy
        (r'(?P<date>(?P<day>(0?\d)|([12]\d)|(3[01]))((st)|(th)|(nd)|(rd))? (?P<month>'+ m + ')(-|([.,]? ))(?P<year>((19)|(20))?\d{2}))', 125, 193), # day month year
        (r'(?P<date>(?P<month>'+ m + ')(-|(\.? ))(?P<day>(0?\d)|([12]\d)|(3[01]))((st)|(th)|(nd)|(rd))?(-|(,? ))(?P<year>((19)|(20))?\d{2}))', 194, 227), # month day year
        (r'(?P<date>(?P<month>'+ m + ')(([.,]? )|/)(?P<year>((19)|(20))\d{2}))', 228, 342), # month year
        (r'(?P<date>(?P<month>(0?\d)|(1[012]))[/-](?P<year>((19)|(20))?\d{2}))', 343, 454), # mm/yy
        (r'(?P<date>(?P<year>((19)|(20))\d{2}))', 455, 499) # just year
    ]

    extracted = pd.concat([df.loc[l:r].str.extract(regex) for regex, l, r in regex_and_range], sort=False)

    extracted.day.fillna('1', inplace=True)
    extracted.day.replace(to_replace=r'^(\d)$', value='0\\1', regex=True, inplace=True)

    extracted.month.fillna('1', inplace=True)
    months_to_replace = {}
    for idx in range(1, 13):
        months_to_replace[calendar.month_abbr[idx]] = str(idx)
        months_to_replace[calendar.month_name[idx]] = str(idx)
    extracted.month.replace(to_replace=months_to_replace, inplace=True)
    extracted.month.replace(to_replace=r'^(\d)$', value='0\\1', regex=True, inplace=True)
    
    extracted.year.replace(to_replace=r'^(\d\d)$', value='19\\1', regex=True, inplace=True)
    
    extracted['dt'] = pd.to_datetime(extracted.apply(lambda x: "{}-{}-{}".format(x['year'], x['month'], x['day']), axis=1))
    
    extracted['orig_index'] = np.arange(0, len(df))
    extracted_sorted = extracted.sort_values(by='dt')[['orig_index', 'dt']]
    extracted_sorted.set_index(np.arange(0, len(df)), inplace=True)
    return extracted_sorted['orig_index']

date_sorter()

0        9
1       84
2        2
3       53
4       28
5      474
6      153
7       13
8      129
9       98
10     111
11     225
12      31
13     171
14     191
15     486
16     335
17     415
18      36
19     405
20     323
21     422
22     375
23     380
24     345
25      57
26     481
27     436
28     104
29     299
      ... 
470    220
471    208
472    243
473    139
474    320
475    383
476    244
477    286
478    480
479    431
480    279
481    198
482    381
483    463
484    366
485    439
486    255
487    401
488    475
489    257
490    152
491    235
492    464
493    253
494    427
495    231
496    141
497    186
498    161
499    413
Name: orig_index, Length: 500, dtype: int64